# Fine-Tuning de VLM (LayoutLMv3) para Extração de Dados de Notas Fiscais

Este notebook demonstra o processo completo de ajuste fino (fine-tuning) de um Modelo de Linguagem e Visão (VLM), especificamente o **LayoutLMv3**, para a tarefa de extração de informações de documentos.

**Objetivo:** Treinar um modelo capaz de identificar e extrair entidades específicas (como cabeçalhos, perguntas, respostas) de imagens de formulários.

**Dataset de Exemplo:** Usaremos o dataset `FUNSD` como um substituto prático para um dataset de Notas Fiscais. Ele contém imagens de formulários e suas respectivas anotações (palavras, coordenadas e rótulos), que é exatamente a estrutura necessária. **O fluxo de trabalho apresentado aqui é 100% aplicável a um dataset customizado de Notas Fiscais.**

**Ambiente:** Recomenda-se executar este notebook no Google Colab com um acelerador de GPU ativado (`Ambiente de execução` > `Alterar o tipo de ambiente de execução` > `GPU`).

## Passo 0: Instalação das Dependências

Primeiro, instalamos todas as bibliotecas necessárias da Hugging Face, além de ferramentas auxiliares para processamento de imagem e avaliação de métricas.

In [ ]:
# Este bloco instala todas as bibliotecas necessárias da Hugging Face e outras
# ferramentas que usaremos para processamento de imagem e avaliação.
!pip install -q transformers datasets Pillow seqeval accelerate "evaluate[seqeval]"

## Passo 1: Carregamento e Preparação do Dataset

Carregamos o dataset de exemplo e preparamos os rótulos (entidades) que queremos que o modelo aprenda a identificar. Para um projeto real, esta é a etapa onde você carregaria suas próprias imagens e anotações (por exemplo, exportadas do Label Studio).

In [ ]:
from datasets import load_dataset

# Carregamos o dataset do Hugging Face Hub
print(">>> Carregando o dataset de exemplo (FUNSD)...")
dataset = load_dataset("nielsr/funsd")

# Vamos inspecionar os rótulos (labels) do dataset
labels = dataset["train"].features["ner_tags"].feature.names
print(f"\nOs rótulos (entidades) neste dataset são: {labels}")

# Criamos mapeamentos de ID para rótulo e vice-versa.
# Você fará o mesmo para suas entidades (ex: 'cnpj_emitente', 'valor_total').
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

print(f"\nMapeamento id2label: {id2label}")
print(f"Mapeamento label2id: {label2id}")

## Passo 2: Pré-processamento dos Dados

Esta é uma etapa crucial. Convertemos as imagens, textos e suas coordenadas para o formato exato que o modelo LayoutLMv3 espera como entrada. O `AutoProcessor` da Hugging Face simplifica enormemente este processo, lidando com o OCR implícito, tokenização e alinhamento de rótulos.

In [ ]:
from transformers import AutoProcessor
from PIL import Image

# Carregamos o processador do modelo. Ele lida com a tokenização do texto
# e o processamento da imagem (OCR implícito e normalização).
model_checkpoint = "microsoft/layoutlmv3-base"
processor = AutoProcessor.from_pretrained(model_checkpoint, apply_ocr=True)

def preprocess_data(examples):
    # Pega as imagens, palavras e caixas delimitadoras
    images = [Image.open(path).convert("RGB") for path in examples['image_path']]
    words = examples['words']
    boxes = examples['bboxes']
    word_labels = examples['ner_tags']

    # Usa o processador para tokenizar as palavras e preparar as imagens
    encoded_inputs = processor(images, words, boxes=boxes, word_labels=word_labels,
                               padding="max_length", truncation=True)

    return encoded_inputs

# Aplicamos a função de pré-processamento a todo o dataset
print("\n>>> Iniciando o pré-processamento dos dados...")
processed_train_dataset = dataset["train"].map(
    preprocess_data, 
    batched=True, 
    remove_columns=dataset["train"].column_names,
    features=dataset["train"].features
)
processed_eval_dataset = dataset["test"].map(
    preprocess_data, 
    batched=True, 
    remove_columns=dataset["test"].column_names, 
    features=dataset["test"].features
)

# Define o formato para PyTorch, que usaremos para o treinamento
processed_train_dataset.set_format(type="torch")
processed_eval_dataset.set_format(type="torch")

print("\nPré-processamento concluído!")
print(f"Exemplo de uma amostra processada (chaves): {processed_train_dataset[0].keys()}")

## Passo 3: Fine-Tuning do Modelo

Agora, configuramos e executamos o treinamento. Carregamos o modelo `LayoutLMv3ForTokenClassification` pré-treinado, definimos os parâmetros de treinamento (como taxa de aprendizado, número de épocas) e uma função para calcular as métricas de performance (F1, precisão, recall). O `Trainer` da Hugging Face gerencia todo o ciclo de treinamento para nós.

In [ ]:
from transformers import LayoutLMv3ForTokenClassification, TrainingArguments, Trainer
import evaluate
import numpy as np

# Carregamos o modelo pré-treinado, especificando nossos rótulos customizados
model = LayoutLMv3ForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id
)

# Carregamos a métrica 'seqeval' para avaliar a performance na extração de entidades
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Removemos os tokens especiais ([CLS], [SEP], [PAD]) para a avaliação
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Definimos os argumentos de treinamento
training_args = TrainingArguments(
    output_dir="meu-modelo-layoutlmv3-notas-fiscais",
    num_train_epochs=3, # Usamos 3 épocas para um teste rápido e eficaz
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=3e-5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=False,
)

# Instanciamos o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_train_dataset,
    eval_dataset=processed_eval_dataset,
    compute_metrics=compute_metrics,
)

# Inicia o treinamento! (Isso pode levar de 20 a 40 minutos na GPU do Colab)
print("\n>>> Iniciando o Fine-Tuning. Isso levará alguns minutos...")
trainer.train()

## Passo 4: Salvando o Modelo Final

Após o treinamento, salvamos o melhor modelo e o processador em um diretório local. Esses arquivos contêm tudo o que é necessário para usar o modelo posteriormente para inferência.

In [ ]:
print("\n>>> Salvando o modelo treinado localmente...")
final_model_path = "modelo-notas-fiscais-final"
trainer.save_model(final_model_path)
processor.save_pretrained(final_model_path)
print(f"Modelo salvo na pasta '{final_model_path}'!")

## Passo 5: Inferência e Visualização

Esta é a etapa final, onde usamos nosso modelo treinado para extrair informações de uma nova imagem. Criamos uma função que processa a imagem, passa pelo modelo e retorna as entidades detectadas e suas coordenadas. Para um resultado mais claro, visualizamos essas detecções desenhando caixas na imagem original.

In [ ]:
import torch
from transformers import AutoModelForTokenClassification
from PIL import Image, ImageDraw

print("\n>>> Executando inferência em uma nova imagem...")

# Carrega o modelo e processador salvos
model_final = AutoModelForTokenClassification.from_pretrained(final_model_path)
processor_final = AutoProcessor.from_pretrained(final_model_path)

# Função para realizar a inferência e extrair as entidades
def extract_entities(image, model, processor):
    # Prepara a imagem
    encoding = processor(image, return_tensors="pt")
    # Move os tensores para a GPU se disponível
    if torch.cuda.is_available():
        for k,v in encoding.items():
            encoding[k] = v.to(model.device)
    
    # Faz a predição
    with torch.no_grad():
        outputs = model(**encoding)

    # Pega as predições e as coordenadas
    logits = outputs.logits
    predictions = logits.argmax(-1).squeeze().tolist()
    boxes = encoding["bbox"].squeeze().tolist()
    
    # Mapeia as predições de volta para os rótulos
    results = []
    for pred_id, box in zip(predictions, boxes):
        label = model.config.id2label[pred_id]
        if label != 'O': # Ignora tokens que não são entidades ('Other')
            results.append({
                "label": label,
                "box": [int(c) for c in box] # Converte coordenadas para inteiros
            })
    return results

# Pega uma imagem do dataset de teste para simular uma "nova" nota fiscal
image_to_test = Image.open(dataset["test"][0]['image_path']).convert("RGB")

# Extrai as entidades da imagem de teste
extracted_data = extract_entities(image_to_test, model_final, processor_final)

print("\n--- DADOS EXTRAÍDOS ---")
print(extracted_data)

# Visualização dos resultados
draw = ImageDraw.Draw(image_to_test)
label_color_map = {'QUESTION': 'blue', 'ANSWER': 'green', 'HEADER': 'orange'}

for data in extracted_data:
    box = data['box']
    label = data['label'].split('-')[-1] # Pega o rótulo base (ex: B-ANSWER -> ANSWER)
    color = label_color_map.get(label, 'red') # Default para vermelho se não mapeado
    draw.rectangle(box, outline=color, width=2)
    
print("\nExibindo imagem com as entidades detectadas:")
image_to_test.save("resultado_inferencia.png")
display(image_to_test)

## Próximos Passos

1.  **Crie seu Dataset:** Colete suas imagens de Notas Fiscais e use uma ferramenta como o **Label Studio** para anotar as entidades que deseja extrair (ex: `cnpj_emitente`, `valor_total`, `data_emissao`, etc.).

2.  **Adapte o Código:**
    - Modifique o **Passo 1** para carregar seus dados customizados.
    - Atualize as variáveis `labels`, `id2label`, e `label2id` com suas próprias entidades.

3.  **Treine o Modelo:** Execute o notebook com seus dados. O resultado será um modelo especialista em extrair informações das suas Notas Fiscais.